In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns
import pickle
from imblearn.over_sampling import ADASYN
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import math

In [2]:
df = pd.read_csv('pre_encoding_data.csv')

In [3]:
df = df.drop(['prod_grp','sub_class','brand', 'dosage', 'dosage_form', 'per_pack_qty','line_qty','pack_price', 'unit_price',
              'first_line','actual_del_year', 'delay','actual_del_month', 'actual_del_day','total_cost','ship_mode'], axis=1)

In [4]:
cat_feats = df.dtypes[df.dtypes == 'object'].index.tolist()
num_feats = df.dtypes[~df.dtypes.index.isin(cat_feats)].index.tolist()

#Convert cat_feats to categorical
for col in cat_feats:
    df[col] = df[col].astype('str')

# Removing delay from num_feats as it is the dependant variable
num_feats.remove('freight')

In [5]:
# Create Pipeline
numerical = Pipeline([('standard_scaler', StandardScaler())])
                            
categorical = Pipeline([('encode', OneHotEncoder()),('Dense',FunctionTransformer(csr_matrix.todense))])

preprocess = ColumnTransformer([('categorical', categorical, cat_feats),('numerical', numerical, num_feats)])

In [6]:
X = df.drop(columns=['freight'])
y = df['freight']

In [7]:
#Fit Preprocess to data
X = preprocess.fit_transform(X)

In [8]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=50)
X_train.shape

(7473, 367)

In [9]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
hide1 = tf.keras.layers.Dense(256, activation='relu')(inputs)
hide2 = tf.keras.layers.Dense(256, activation='relu')(hide1)
hide3 = tf.keras.layers.Dense(256, activation='relu')(hide2)
hide4 = tf.keras.layers.Dense(128, activation='relu')(hide3)
hide5 = tf.keras.layers.Dense(64, activation='relu')(hide4)
outputs = tf.keras.layers.Dense(1, activation='linear')(hide5)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mean_absolute_error'])

history = model.fit(X_train,y_train,validation_split=0.2,batch_size=32,epochs=200)
    #callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

Epoch 1/200
187/187 [==============================] - 3s 6ms/step - loss: 8191.1484 - mean_absolute_error: 8191.1484 - val_loss: 6458.7681 - val_mean_absolute_error: 6458.7681
Epoch 2/200
187/187 [==============================] - 1s 5ms/step - loss: 6553.2178 - mean_absolute_error: 6553.2178 - val_loss: 5686.2627 - val_mean_absolute_error: 5686.2627
Epoch 3/200
187/187 [==============================] - 1s 5ms/step - loss: 6006.8252 - mean_absolute_error: 6006.8252 - val_loss: 5217.6226 - val_mean_absolute_error: 5217.6226
Epoch 4/200
187/187 [==============================] - 1s 7ms/step - loss: 5566.1396 - mean_absolute_error: 5566.1396 - val_loss: 5268.9648 - val_mean_absolute_error: 5268.9648
Epoch 5/200
187/187 [==============================] - 1s 8ms/step - loss: 5279.9199 - mean_absolute_error: 5279.9199 - val_loss: 4762.4277 - val_mean_absolute_error: 4762.4277
Epoch 6/200
187/187 [==============================] - 1s 6ms/step - loss: 5036.1758 - mean_absolute_error: 5036.17

In [10]:
# Make predictions and calculate the scores
y_pred = (model.predict(X_test))

print(mean_squared_error(y_test, y_pred))
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(mean_absolute_error(y_test,y_pred))
print(r2_score(y_test, y_pred))

60428280.85237518
7773.562944517474
3374.030014731784
0.7993324451480163


## XGBRegressor

In [12]:
from xgboost import XGBRegressor
reg = XGBRegressor(n_estimators=1000, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
f = reg.fit(X_train, y_train)

In [13]:
y_pred = (f.predict(X_test))

print(mean_squared_error(y_test, y_pred))
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(mean_absolute_error(y_test,y_pred))
print(r2_score(y_test, y_pred))

49545776.58012623
7038.8760878514
3249.9926088519524
0.8354705826587183


## Hyperparameter Tuning

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
params = { 'max_depth': [5,8,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [500,700, 1000],
           'colsample_bytree': [0.3, 0.5]}
reg = GridSearchCV(estimator=XGBRegressor(),
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=1)
reg.fit(X, y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_esti...
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=N

In [16]:
print("Best parameters:", reg.best_params_)
print("Lowest RMSE: ", (-reg.best_score_)**(1/2))

Best parameters: {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 1000}
Lowest RMSE:  7810.284691283787


In [17]:
best_model = reg.best_estimator_

In [18]:
y_pred = (best_model.predict(X_test))

print(mean_squared_error(y_test, y_pred))
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(mean_absolute_error(y_test,y_pred))
print(r2_score(y_test, y_pred))

919930.304103604
959.129972476934
495.755567621859
0.9969451362482132


In [19]:
# Save the best model
pickle.dump(best_model, open('best_freight_model.pkl', 'wb'))